In [1]:
from pyprojroot.here import here
import sys
import pandas as pd
import pathlib
from collections import defaultdict, Counter
import copy

sys.path.append(str(here()))

from resources.utils import *
from test_utils import *

In [4]:
def main(dry_run=True,
         resources=None,
         output_resources='template_user/resources/resources.yml'):
    """
    Generate resources.yml with path_map and user list.
    Copies template_user dir for each user.
    Supports dry-run mode for safe testing.

    Parameters
    ----------
    dry_run : bool
        If True, no destructive operations are performed; operations are logged.
    resources : dict | str | None
        Path to resources.yml or a pre-loaded dict.
    output_config : str | Path | None
        Path to write the generated resources.yml. Defaults to
        'template_user/resources/resources.yml'.
    """
    
    m = load_yml(resources)
    
    # make sure project name has been changed
    proj_name = m['setup_settings']['project_name']
    verify_proj_name(proj_name)
    
    # make sure usernames are unique
    usernames = [i2['username']
        for _, i in m['setup_settings']['users'].items()
        for _, i2 in i.items()]    
    check_setup_usernames(usernames)
    
    # destructive operations
    safe_run("rm -rf .git", dry_run=dry_run)
    safe_run(f"mv ../project_template ../{proj_name}", dry_run=dry_run)

    path_map = generate_path_map(m['setup_settings'], proj_name)
    
    # also add a users list
    users_list = {'users': list(m['setup_settings']['users'].keys())}
    
    # new user's folder
    dest = Path(user_alias)

    # write path map and users to resources.yml
    if dry_run:
        # return the generated data for testing
        print(f"[DRY-RUN] Would append YAML to {output_resources}")
        return {'path_map': path_map, 'users': users_list}
        print(f"[DRY-RUN] Would copy {curr_user_dir} -> {new_user_dir}")
    
    else:
        with Path(output_resources).open('a') as f:
            yaml.dump({'path_map': path_map}, f, default_flow_style=False)
            yaml.dump(users_list, f, default_flow_style=False)

        # copy template_user for each user
        for user_alias in m['setup_settings']['users']:
            shutil.copytree("template_user", dest, dirs_exist_ok=True)

        return None

In [5]:
m = main(dry_run=True, 
         resources='test_yml.yml',
        output_resources='test_yml_out.yml')

[DRY-RUN] Would execute: rm -rf .git
[DRY-RUN] Would execute: mv ../project_template ../my_test_project
[DRY-RUN] Would write YAML to test_yml_out.yml


In [6]:
m

{'path_map': {'bscuser1': {'new_dir': '/new/mn5/dir/user1',
   'projects_dir': '/gpfs/projects/bsc83',
   'scratch_dir': '/gpfs/scratch/bsc83',
   'data_dir': '/gpfs/projects/bsc83/Projects/my_test_project/data',
   'ref_dir': '/gpfs/projects/bsc83/Projects/my_test_project/ref',
   'figures_dir': '/gpfs/projects/bsc83/Projects/my_test_project/figures',
   'metadata_dir': '/gpfs/projects/bsc83/Projects/my_test_project/user1_alias/metadata'},
  'mn5_user': {'new_dir': '/new/mn5/dir/user1',
   'projects_dir': '/gpfs/projects/bsc83',
   'scratch_dir': '/gpfs/scratch/bsc83',
   'data_dir': '/gpfs/projects/bsc83/Projects/my_test_project/data',
   'ref_dir': '/gpfs/projects/bsc83/Projects/my_test_project/ref',
   'figures_dir': '/gpfs/projects/bsc83/Projects/my_test_project/figures',
   'metadata_dir': '/gpfs/projects/bsc83/Projects/my_test_project/user2_alias/metadata'},
  'localuser1': {'scratch_dir': '/user1/sshfs/scratch',
   'projects_dir': '/user1/sshfs/projects',
   'new_dir': '/user1/